In [62]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd
import numpy as np
import requests
import re
from datetime import datetime

In [63]:
now = datetime.now()
options = webdriver.ChromeOptions() 
options.add_experimental_option("excludeSwitches", ['enable-automation']) # allows us to scrape bloomberg
tickers = ['APPL']

In [66]:
def bloombergSearch(ticker):
    url = 'https://www.bloomberg.com/search?query=%s'%(ticker)
    browser = webdriver.Chrome(chrome_options=options, executable_path=r"/usr/bin/chromedriver") # path should be similar for ubuntu vm
    browser.get(url)
    time.sleep(5) # wait for the page to load javascript
    pageSource = browser.page_source
    browser.quit()
    return pageSource

def bloombergArticle(url):
    browser = webdriver.Chrome(chrome_options=options, executable_path=r"/usr/bin/chromedriver")
    browser.get(url)
    time.sleep(5) # wait for the page to load javascript
    pageSource = browser.page_source
    browser.quit()
    return pageSource

def contentCat(content):
# concatenate content strings of article
    article = ''
    for par in content:
        if par == content[-1]:
            article += par.get_text()
        else:
            article += par.get_text() + '\n'
    return article

def scrapeSearch(pageSource):
    soup = BeautifulSoup(pageSource, 'html.parser')
    searchResults = soup.select('div>div>section:nth-child(3)>div:last-child>div>a', id='root')
    return searchResults

def scrapeArticle(pageSource):
    soup = BeautifulSoup(pageSource, 'html.parser')
    url = soup.select('head>title+link')[0]['href']
    title = soup.find_all('h1')[0].get_text()
    author = soup.find_all(class_='author-v2__byline')[0].get_text()
    posted = soup.find_all('time', class_='article-timestamp')[0].get_text() # time of posting
    edited = soup.find_all('time', class_='article-timestamp')[1].get_text() # time if edited
    scraped = now.strftime("%d/%m/%Y %H:%M:%S")
    content = soup.select('div>p', class_='body-copy-v2')
    article = contentCat(content)
    return [url, title, author, posted, edited, scraped, article]

In [24]:
searchSource = bloombergSearch('AAPL')
searchResults = scrapeSearch(searchSource)

In [39]:
articleResultsRaw = []
for link in searchResults[:3]:
    articleSource = bloombergArticle(link['href'])
    articleResultsRaw.append(articleSource)

/home/albert/anaconda3/envs/cs/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


In [67]:
articleResults = []
for article in articleResultsRaw: 
    articleResults.append(scrapeArticle(article))

In [70]:
df = pd.DataFrame(articleResults, columns=['URL', 'title', 'author', 'postedDate', 'editedDate', 'scrapedDate', 'content'])

In [72]:
df.to_pickle('articles.pkl')

In [73]:
df

,URL,title,author,postedDate,editedDate,scrapedDate,content
0,https://www.bloomberg.com/news/articles/2019-0...,Apple’s New Mac Pro to Be Assembled in Texas A...,Mark Gurman,"September 23, 2019, 12:00 PM EDT","September 23, 2019, 12:00 PM EDT",27/09/2019 03:06:38,Apple Inc. said the next version of its high-e...
1,https://www.bloomberg.com/news/articles/2019-0...,‘More Buyers Than Sellers’ Assumes a New Identity,Brad Olesen,"September 24, 2019, 7:36 AM EDT","September 24, 2019, 7:36 AM EDT",27/09/2019 03:06:38,Tuesday couldn’t be more welcome coming off a ...
2,https://www.bloomberg.com/news/articles/2019-0...,World’s Biggest ETF Defies Stock Unease With $...,Rachel Evans,"September 10, 2019, 7:01 AM EDT","September 10, 2019, 11:13 AM EDT",27/09/2019 03:06:38,Traders are going all-in on stocks via the wor...


In [74]:
# get ticker stock data
def stockData(ticker):
    url = 'https://finance.yahoo.com/quote/%s'%(ticker)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    summaryTable = soup.select('div div table tbody tr td:nth-child(2)', id='quote-summary')
    previous_close = summaryTable[0].get_text()
    return [previous_close]

In [50]:
CGN = stockData('CGN.SI')